In [5]:
import logging
import json


from pymongo import MongoClient
from importlib import resources
import os
import re
import datetime
import glob

from codex.database.vasp import generate_json as vasp_generate_json
from codex.database.espresso import run_helpdoc
from codex.database.vasp import generate_json as espresso_generate_json

logging.basicConfig(level=logging.WARNING)

In [6]:
client = MongoClient('localhost', 27017)

In [5]:
import re
CDX_ID_ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
CDX_ID_LENGTH = 12
cdxid = 'cdx-c-fvr9MlHqeTqw'
CDX_ID_ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
CDX_ID_LENGTH = 12
CDX_ID_REGEX = fr"cdx-([fcp])-[{CDX_ID_ALPHABET}]{{{CDX_ID_LENGTH}}}"
if not re.match(CDX_ID_REGEX, cdxid):
    raise ValidationError("Invalid cdxid")
cdxid.split("-")[1]

'c'

In [8]:
from codex.app.models import AbstractFileCodex, VaspFileCodex, EspressoFileCodex

In [13]:
issubclass(type(espresso_codex), AbstractFileCodex)

True

In [10]:
from codex.app.models.vasp import VaspFileCodex
from codex.app.models.espresso import EspressoFileCodex
#from codex.api.entry_schemas import CodexEntrySchema
from werkzeug.datastructures import FileStorage
import os

with open('example/espresso/bi4te3_bandsx.in', 'rb') as f:
    f = FileStorage(f)
    f.filename = os.path.basename(f.filename)
    espresso_codex = EspressoFileCodex.from_file(f, client, dbversion='latest')
with open('example/vasp/INCAR_Fe3O4.vasp', 'rb') as f:
    f = FileStorage(f)
    f.filename = os.path.basename(f.filename)
    vasp_codex = VaspFileCodex.from_file(f, client, dbversion='latest')
##client.drop_database('codex')
#result = client['codex']['entries'].insert_many(CodexEntrySchema().dump([codex1, codex2], many=True))
##print(result.inserted_id)

In [11]:
espresso_codex.to_string()

"&bands\n  prefix      = 'bi4te3'               ! of files saved by program pw\n  outdir      = './out/'               ! directory containing the input data, i\n  lsigma(1:3) = .TRUE., .TRUE., .TRUE. ! If true computes expectation values of the spin operator on the spinor wave\n  lsym        = .FALSE.                ! If\n  no_overlap  = .FALSE.                ! If\n/"

In [7]:
print(espresso_codex.to_string(pretty=True, with_comments=True))

&bands
  prefix      = 'bi4te3'               ! of files saved by program pw
  outdir      = './out/'               ! directory containing the input data, i
  lsigma(1:3) = .TRUE., .TRUE., .TRUE. ! If true computes expectation values of the spin operator on the spinor wave
  lsym        = .FALSE.                ! If
  no_overlap  = .FALSE.                ! If
/


In [9]:
vasp_codex._id

'316aa8d3d84b43669877cbc51305c899'

In [13]:
import requests
URL = 'http://localhost:5000/calcs/cdx-c-fvr9MlHqeTqw'
requests.get(URL).json()

{'_id': 'cdx-c-fvr9MlHqeTqw',
 'code': 'espresso',
 'created': '2023-06-28T01:18:51.811683+00:00',
 'dbversion': '7.2',
 'file_ids': ['cdx-f-bfbqnuh9YalR',
  'cdx-f-ovaXqHgcJVKw',
  'cdx-f-5nOexMABjVJ4']}

In [10]:
espresso_codex._id

'316aa8d3d84b43669877cbc51305c899'

In [4]:
from codex.models import CodexTag, FILE_CODEX_MAP
from datetime import datetime, timezone

from marshmallow import Schema, fields, post_load
class CodexTagSchema(Schema):
    """Schema for a Codex's tags"""

    name = fields.String(required=True)
    value = fields.String(required=True)
    comment = fields.String(required=True)
    tag_pad = fields.String(required=True)
    value_pad = fields.String(required=True)
    comment_pad = fields.String(required=True)
    id = fields.String(required=True)
    href = fields.String(required=True)
    indent = fields.String(required=True)

    @post_load
    def create_tag(self, data, **kwargs):
        """Deserialize as object"""
        return CodexTag(**data)

class CodexEntrySchema(Schema):
    """Schema for Codex objects"""

    _id = fields.UUID(required=True)
    created = fields.DateTime(required=True, dump_default=datetime.now(timezone.utc))

    tags = fields.Dict(
        keys=fields.Str(), values=fields.List(fields.Nested(CodexTagSchema)), required=True
    )
    cards = fields.String(required=True)
    filename = fields.String(required=True)
    raw_file = fields.String(required=True)
    filetype = fields.String(required=True)
    code = fields.String(required=True)
    dbversion = fields.String(required=True)
    comment_token = fields.String(required=True)
    section_start_token = fields.String(required=True)
    section_end_token = fields.String(required=True)
    indent = fields.Integer(required=True)

    # TODO: this shouldn't be necessary
    @post_load
    def create_codex(self, data, **kwargs):
        """Deserialize as object"""
        Codex = FILE_CODEX_MAP[data["code"]]
        return Codex(**data)

In [5]:
dump = CodexEntrySchema().dump(vasp_codex)
load = CodexEntrySchema().load(dump)

Got kwargs {'cards': '', 'tags': {'': [<codex.models.codex.CodexTag object at 0x10b1bb190>, <codex.models.codex.CodexTag object at 0x137742050>, <codex.models.codex.CodexTag object at 0x10b1bb790>, <codex.models.codex.CodexTag object at 0x108f48b10>, <codex.models.codex.CodexTag object at 0x108f49b90>, <codex.models.codex.CodexTag object at 0x1088ff910>, <codex.models.codex.CodexTag object at 0x13771d790>, <codex.models.codex.CodexTag object at 0x13771c910>, <codex.models.codex.CodexTag object at 0x13771e050>, <codex.models.codex.CodexTag object at 0x13771fd50>, <codex.models.codex.CodexTag object at 0x13771c950>, <codex.models.codex.CodexTag object at 0x13771df50>, <codex.models.codex.CodexTag object at 0x13771c190>, <codex.models.codex.CodexTag object at 0x13771e4d0>, <codex.models.codex.CodexTag object at 0x13771c250>, <codex.models.codex.CodexTag object at 0x13771e390>, <codex.models.codex.CodexTag object at 0x13771e450>, <codex.models.codex.CodexTag object at 0x13771e250>, <codex.

In [6]:
load

In [9]:
all((dump['tags'], dump['cards'], dump['filetype']))

False

In [6]:
dump = CodexTagSchema().dump(espresso_codex.tags['bands'], many=True)
loads = CodexTagSchema().load(dump, many=True)

In [8]:
loads[0].to_string()

"  prefix      = 'bi4te3'               ! of files saved by program pw"

In [4]:
espresso_codex.tags['bands'][0].to_string()

"  prefix      = 'bi4te3'               ! of files saved by program pw"

In [55]:
from codex.api.file_schemas import FileCodexQueryArgsSchema
params = {'filetype': 'INCAR', 'code': 'vasp'}
#params = {'_id': ['e44e5a84f8034db1a9fab5b391c2cd34', 'dff217b9190e4d1db5d118d09d0463f7']}
#print(CodexEntryQueryArgsSchema().dump(params))
x = requests.get(URL, params=params).json()
for xx in x:
    print(xx['_id'])

3954583e-94ba-41df-90f3-86d673ce5189
83e594ee-ad85-4c81-b72f-ffa53d8ce9a4
dff217b9190e4d1db5d118d09d0463f7
4b364556c1954f32b15a5fbdc52511b9
59f5fb0ea20840878cab45f8e1eb2c0f
d606e50af4bf49fbb3344e666b3a0d69
e44e5a84f8034db1a9fab5b391c2cd34


In [202]:
import requests

URL = "http://localhost:5000/collections/fromfile"
data = {"dbversion": "latest", "code": "vasp"}
filename1 = "example/vasp/POSCAR_Fe3O4.vasp"
filename2 = "example/vasp/INCAR_Fe3O4.vasp"
#filename1 = "example/espresso/Bi2Te3_proj.in"
#filename2 = "example/espresso/Si_bands.in"
files = [
    ("input_file", (filename1, open(filename1, "rb"))),
    ("input_file", (filename2, open(filename2, "rb"))),
    #("dbversion", (None, 'latest')),
    #("code", (None, 'vasp')),
]
x = requests.post(URL, files=files, params=data)
#x.json()

In [203]:
x.json()

{'_id': 'cdx-GJLACd89W2sl',
 'code': 'vasp',
 'created': '2023-06-26T23:50:49.592886+00:00',
 'dbversion': '1687467268',
 'entry_ids': ['cd0db0c5163e4a1fbdc4d988e9a69099',
  '4d0049d6d5564636afd686592f43ef3d']}

In [180]:
URL = "http://localhost:5000/collections/"
cdxid="cdx-4JB6Bs1wTu9x"
x = requests.get(URL+cdxid)

In [201]:
ids = ['d310e32060504edbb1029a0739423884', 'd146022a653743069953ef440402717c']
result = client['codex']['entries'].find({'_id': {'$in': ids}})
result.count_documents()

AttributeError: 'Cursor' object has no attribute 'count_documents'

In [200]:
len(list(result))

2

In [182]:
x.json()

{'_id': 'cdx-4JB6Bs1wTu9x',
 'code': 'vasp',
 'created': '2023-06-26T22:29:01.847784+00:00',
 'dbversion': '1687467268',
 'entry_ids': ['a3d6d7f56e6843a19147ed6f923ed1c0',
  '635d50aa3a7f411d86c31542b9b45320']}

In [162]:
x.json()

{'_id': 'cdx-PIHsL9HEXECR',
 'code': 'espresso',
 'created': '2023-06-26T22:14:35.813497+00:00',
 'dbversion': '7.2',
 'entry_ids': ['38610fd0eaf048479527832ac7d1bfa8',
  'e994f0d4601f496e888698afce47e5b8']}

In [160]:
URL = "http://localhost:5000/collections/"
cdxid = 'cdx-l9kO2ddFJf7B'
url = URL+cdxid
x = requests.get(url)

In [159]:
x.json()

{'_id': 'cdx-l9kO2ddFJf7B',
 'code': 'espresso',
 'created': '2023-06-26T21:43:10.456670+00:00',
 'dbversion': '7.2',
 'entries': [{'_id': '23fc66ffcc974d9cb705a8efbad740d2',
   'cards': '',
   'code': 'espresso',
   'code_pretty': 'Quantum ESPRESSO',
   'comment_token': '! ',
   'created': '2023-06-26T21:43:10.456355+00:00',
   'filename': 'Bi2Te3_proj.in',
   'filetype': 'projwfc',
   'pad': ' ',
   'section_end_token': '/',
   'section_start_token': '&',
   'tags': {'projwfc': [{'comment': 'of input file produced by pw',
      'comment_pad': 10,
      'id': 'prefix',
      'name': 'prefix',
      'tag_pad': 2,
      'value': "'bi2te3'",
      'value_pad': 1},
     {'comment': 'directory containing the input data, i',
      'comment_pad': 13,
      'id': 'outdir',
      'name': 'outdir',
      'tag_pad': 2,
      'value': "'out'",
      'value_pad': 1},
     {'comment': 'prefix for output files containing PDOS',
      'comment_pad': 2,
      'id': 'filpdos',
      'name': 'filpdos',


In [210]:
requests.get('http://127.0.0.1:5000/entries/16a805609f3148f68c218850e5d738d5').json()

{'_id': '16a805609f3148f68c218850e5d738d5',
 'cards': '',
 'code': 'vasp',
 'comment_token': '! ',
 'created': '2023-06-27T08:24:34.096480+00:00',
 'dbversion': '1687467268',
 'filename': 'INCAR_Fe3O4.vasp',
 'filetype': 'INCAR',
 'indent': 0,
 'pad': ' ',
 'section_end_token': '',
 'section_start_token': '',
 'tags': {'': [{'comment': 'The "title string" defined by SYSTEM is for the user only and should help the user to identify what he/she wants to do with this specific input file.',
    'comment_pad': 10,
    'id': 'SYSTEM',
    'name': 'SYSTEM',
    'tag_pad': 3,
    'value': 'Supercell',
    'value_pad': 1},
   {'comment': 'NSCF calculation, read from CHGCAR',
    'comment_pad': 17,
    'id': 'ICHARG',
    'name': 'ICHARG',
    'tag_pad': 3,
    'value': '11',
    'value_pad': 1},
   {'comment': 'specifies the total number of KS or QP orbitals in the calculation.',
    'comment_pad': 16,
    'id': 'NBANDS',
    'name': 'NBANDS',
    'tag_pad': 3,
    'value': '192',
    'value_pad

In [118]:
list(client['codex']['entries'].find())[20]['created']

'2023-06-26T21:03:49.944628+00:00'

In [145]:
from datetime import datetime, timezone
timestamp = datetime.now(timezone.utc).isoformat()
print(timestamp)
datetime.fromisoformat(timestamp)

2023-06-26T21:48:12.201933+00:00


datetime.datetime(2023, 6, 26, 21, 48, 12, 201933, tzinfo=datetime.timezone.utc)

In [104]:

from pymatgen.io.vasp.inputs import Incar
with open('example/vasp/INCAR_Fe3O4.vasp', 'r') as f:
    content = f.read()
with tempfile.NamedTemporaryFile() as tmp:
    with open(tmp.name, 'w') as f:
        f.write(content)
    incar = Incar.from_file(tmp.name)
incar

{'SYSTEM': 'Supercell',
 'ICHARG': 11,
 'NBANDS': 192,
 'GGA': 'Ps',
 'ISPIN': 2,
 'MAGMOM': [4, 4, 4, 4, -4, -4, 0, 0, 0, 0, 0, 0, 0, 0],
 'ENCUT': 800,
 'EDIFF': 1e-07,
 'ISIF': 3,
 'EDIFFG': -0.002,
 'IBRION': 2,
 'NSW': 0,
 'AMIN': 0.05,
 'MAXMIX': 40,
 'LASPH': True,
 'LDAU': True,
 'LDAUTYPE': 2,
 'LDAUL': [2, -1],
 'LDAUU': [4.5, 0],
 'LDAUJ': [0, 0],
 'LMAXMIX': 4,
 'ISMEAR': 0,
 'SIGMA': 0.02,
 'LORBIT': 11}

In [36]:
x.request.body

b'--0abbef9aaf04e22dff326c19501e43d8\r\nContent-Disposition: form-data; name="file"; filename="example/vasp/POSCAR_Fe3O4.vasp"\r\n\r\nFe6 O8                                  \n   1.00000000000000     \n     5.9239884022792371    0.0000000000000000    0.0000000000000000\n     2.9619942011396185    5.1303244481330896   -0.0000000000006391\n     2.9619942011396185    1.7101081494098196    4.8369162758792656\n   Fe   O \n     6     8\nDirect\n  0.1250000000000000  0.6250000000000000  0.6250000000000000\n  0.6250000000000000  0.1250000000000000  0.6250000000000000\n  0.6250000000000000  0.6250000000000000  0.1250000000000000\n  0.6250000000000000  0.6250000000000000  0.6250000000000000\n  0.2500000000000000  0.2500000000000000  0.2500000000000000\n  0.0000000000000000  0.0000000000000000  0.0000000000000000\n  0.8698034440847735  0.3905896077456745  0.8698034440847806\n  0.3801965259152240  0.3801965259152169  0.8594103922543255\n  0.3801965259152240  0.3801965259152240  0.3801965259152240\

In [81]:
result = client['codex']['entries'].delete_one({'_id': 'dff217b9190e4d1db5d118d09d0463f7'})

In [88]:
URL = 'http://localhost:5000/entries/d606e50af4bf49fbb3344e666b3a0d69'
requests.delete(URL)

<Response [204]>

In [85]:
import requests
URL = 'http://localhost:5000/entries'
#from codex.api.schemas import CodexEntryQueryArgsSchema
#print(CodexEntryQueryArgsSchema().load(params))
params = {'filetype': 'INCAR'}
x = requests.get(URL, params=params).json()
for xx in x:
    print(xx['_id'])

3954583e-94ba-41df-90f3-86d673ce5189
83e594ee-ad85-4c81-b72f-ffa53d8ce9a4
4b364556c1954f32b15a5fbdc52511b9
59f5fb0ea20840878cab45f8e1eb2c0f
d606e50af4bf49fbb3344e666b3a0d69
e44e5a84f8034db1a9fab5b391c2cd34


In [37]:
import requests
from codex.models.vasp import VaspFileCodex
from codex.models.espresso import EspressoFileCodex
from marshmallow import Schema, fields, validate, ValidationError, pre_dump, pre_load
from codex.api.file_schemas import FileCodexSchema

codex = VaspFileCodex('example/vasp/INCAR_HfTe5.vasp', 'latest', client)
codex1 = VaspFileCodex('example/vasp/POSCAR_Fe3O4.vasp', 'latest', client)
codex2 = EspressoFileCodex('example/espresso/bi4te3_bandsx.in', 'latest', client)
data = FileCodexSchema().dump([codex], many=True)
#print(data.keys())
#double_dump = CodexEntrySchema().dump(data)
#double_dump.keys()

# Add to MongoDB

In [ ]:
base_db_dir = resources.files("codex.database")

json_files = glob.glob(os.path.join(base_db_dir, "json", "*.json"))
for file in json_files:
    insert_json(client, file)

In [ ]:
database_names = client.list_database_names()
available_versions = [
    match.group(1)
    for f in database_names
    for match in [re.match(rf"{code}-([0-9]+\^[0-9]+(?:\^[0-9]+)?)", f)]
    if match
]
print(available_versions)

In [ ]:
get_database(client, 'Quantum ESPRESSO', '7.1')

# VASP database

In [ ]:
db = vasp_generate_json()

# QE database

In [ ]:
versions = ['6.7', '6.8', '7.0', '7.1', '7.2']
database_dir = resources.files("codex.database")

for v in versions:
    #run_helpdoc(v)
    db = espresso_generate_json(v)

# Generate Codex

In [ ]:
from importlib import resources
filename = 'example/sr3pbo_bands.in'
database_dir = resources.files("codex.database")
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
codex = Codex(filename, version)


In [ ]:
print(codex.filename)
print(codex.code)
print(codex.package)
print(codex.tags['control'])

In [ ]:
from importlib import resources
import os
from base64 import b64decode
import json
import shutil

dbversion='7.2'
dir='static'

base_db_dir = resources.files("codex.database")
code = "qe"  # Need better terminology
database_dir = os.path.join(base_db_dir, f"{code}-{dbversion}")
database_filename = os.path.join(database_dir, "database.json")

with open(database_filename) as f:
    database = json.load(f)

working_dir = os.path.abspath(dir)
tags_dir = os.path.join(working_dir, "tags-qe")
if os.path.exists(tags_dir):
    shutil.rmtree(tags_dir, ignore_errors=True)
os.mkdir(tags_dir)
for p, package in database.items():
    print(p)
    os.mkdir(os.path.join(tags_dir, p))
    for nl, namelist in package.items():
        print(nl)
        os.mkdir(os.path.join(tags_dir, p, nl))
        for tag, v in namelist.items():
            print(tag, v)
            if v["html"] != 'No documentation was found for this tag.':
                webpage = b64decode(v["html"]).decode("utf-8")
                path = os.path.join(tags_dir, p, nl, f"{tag}.html")
                with open(path, "w") as f:
                    f.write(webpage)